##Date:  05/25/2015

### This is copied from Ted_5,  use to process the test data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline
import requests
from pattern import web

In [2]:
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk import sent_tokenize
from nltk.corpus import stopwords
import re

In [3]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from unidecode import unidecode
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

##1. Process speakers info 

read in the train.json file

In [4]:
talks_test = pd.read_json('test.json')

In [5]:
talks_test.reset_index(drop = True, inplace = True)

In [7]:
talks_test.shape

(350, 31)

### get all speakers' info 

In [11]:
speakers = pd.read_json('speakers.json')

In [12]:
speakers.columns

Index([  u'description',     u'firstname',            u'id',      u'lastname',
       u'middleinitial',  u'published_at',          u'slug',         u'title',
          u'updated_at',    u'whotheyare',     u'whylisten'],
      dtype='object')

In [14]:
speakers.set_index('id',inplace = True)

In [17]:
speakers.sort_index(inplace = True)

In [20]:
speakers.head()

,description,firstname,lastname,middleinitial,published_at,slug,title,updated_at,whotheyare,whylisten
id,,,,,,,,,,
2,Climate-change prophet,Al,Gore,,2007-02-13 10:43:00,al_gore,,2009-04-08 14:44:57,"Once the US Vice President, then star of An In...","Unlike some in public office, Al Gore always i..."
3,"inventor, engineer",Amy,Smith,,2007-02-13 10:46:00,amy_smith,,2010-06-30 11:15:45,"Amy Smith designs cheap, practical fixes for t...",Mechanical engineer Amy Smith&#39;s approach t...
4,President-elect of Afghanistan,Ashraf,Ghani,,2007-02-13 10:47:00,ashraf_ghani,,2015-03-04 03:50:59,"Ashraf Ghani, Afghanistan’s new president-elec...",Ashraf Ghani became Afghanistan&rsquo;s new pr...
5,Aircraft engineer,Burt,Rutan,,2007-02-13 10:49:00,burt_rutan,,2010-06-15 15:58:14,"In 2004, legendary spacecraft designer Burt Ru...",Burt Rutan is widely regarded as one of the wo...
6,Car designer,Chris,Bangle,,2007-02-13 10:59:00,chris_bangle,,2010-06-15 16:00:03,Car design is a ubiquitous but often overlooke...,American designer Chris Bangle understands tha...


###1.1 Determine gender 

In [21]:
genderdict = {'he':1,'him':1,'his':1,'she':0,'her':0,'hers':0}

In [22]:
def genderdet(text):
    found = False
    for word in text.split(' '):
        word = word.lower()
        if genderdict.has_key(word):
            found = True
            return genderdict[word]
    if not found:
        return -1

In [23]:
# define a function to get speakers info given speakers ids
def speakerinfo(ids):
    # ids is a list of ids
    gender = []
    for speakerid in ids:
        text = speakers.ix[speakerid,'whylisten'] + speakers.ix[speakerid,'whotheyare']
        gender.append(genderdet(text))
    if len(gender) == 1:
        return gender[0]
    return gender

In [24]:
gender = talks_test['speaker_ids'].apply(speakerinfo)

In [31]:
len(gender)

350

#### Resort to gender.api.com to check the remaining speakers using their first name 

In [32]:
# row index of speaker gender-unidentified talks
unidentified = list(gender[gender == -1].index)

In [42]:
unidentified[1]

40

In [36]:
baseurl = 'https://gender-api.com/get?name='
key='oxxyvSjsHaSKyuLUFF'

In [37]:
def checkfirstname(ind):
    # ind is the row indices of gender(talks_train)
    # since in this case we know these talks have only one speaker, we just assign it to speakerid as a variable
    speakerid = talks_test.ix[ind,'speaker_ids'][0]
    name = speakers.ix[speakerid,'firstname']
    url = baseurl+name
    response = requests.get(url,params = dict(key=key))
    return response.json()['gender']

In [43]:
checkfirstname(40)

u'female'

In [44]:
sex = [checkfirstname(x) for x in unidentified]

In [45]:
sex_bin = []
for det in sex:
    if det == 'male':
        sex_bin.append(1)
    elif det == 'female':
        sex_bin.append(0)
    else:
        sex_bin.append(-1)

In [46]:
sex_bin = pd.Series(sex_bin,index = unidentified)

In [47]:
sex_bin

3     -1
40     0
70     1
83     1
100   -1
162    0
166    1
177    0
178    1
189    1
206    1
209    1
239    1
247    1
253    1
305    1
318    1
323   -1
dtype: int64

In [48]:
gender[unidentified]=sex_bin

In [50]:
gender.values

array([1, 1, 0, -1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, [0, 1, 1, 1], 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 1, [0, 1], 0, 1, [0, 1], 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, [-1, 1], 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 

In [51]:
gender[gender == -1].index

Int64Index([3, 100, 323], dtype='int64')

In [62]:
talks_test.ix[170,'name']

u'Sergey Brin + Larry Page: The genesis of Google'

In [56]:
gender[3]=1

In [57]:
gender[100]=0;gender[323]=1

In [60]:
gender.values

array([1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, [0, 1, 1, 1], 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, [0, 1], 0, 1, [0, 1], 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, [-1, 1], 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0,

In [61]:
for ind, sex in gender.iteritems():
    if type(sex)==list:
        if len(sex)==2 and sum(sex)==1:
            gender[ind]=1
        elif len(sex)==3:
            gender[ind]=1
        elif len(sex)==4:
            gender[ind]=1
        else:
            print ind,sex

170 [-1, 1]


In [63]:
gender[170]=1

In [64]:
gender.value_counts()

1    256
0     94
dtype: int64

###gender determination summary: 

1. Most of them are determined by looking for he/she etc in the whytohear, whotheyare columns
2. The unknown from the 1st step are determined by first name check on gendercheck.api.com
3. The remaining unknowns are determined manually by watching the video
4. In the case of multiple speakers: (1) Majority gender (2)If even, choose 'male' or 1
5. Notice there are many cases where there are multiple speakers but TED recorded as one speaker.

In [65]:
gender.shape

(350,)

In [66]:
talks_test['speaker_gender']=gender

In [67]:
talks_test.head()

,Beautiful,Confusing,Courageous,Fascinating,Funny,Informative,Ingenious,Inspiring,Jaw-dropping,Longwinded,...,native_language_code,published_at,rating_count,recorded_at,released_at,speaker_ids,themes,updated_at,viewed_count,speaker_gender
0,23,12,26,430,NaN,238,371,140,377,29,...,en,2011-05-15 12:24:00,1720,2011-03-03,2012-01-03 14:57:43,[890],"[{u'theme': {u'id': 4, u'name': u'How the Mind...",2014-07-03 20:43:21,800492,1
1,22,2,65,36,NaN,129,10,227,40,13,...,en,2011-05-24 14:54:00,674,2011-03-03,2014-06-03 17:19:32,[928],"[{u'theme': {u'id': 18, u'name': u'To Boldly G...",2014-07-03 20:43:21,454366,1
2,122,8,229,82,9,137,5,284,38,17,...,en,2011-05-25 15:49:20,1042,2010-12-04,2014-02-03 22:09:07,[830],"[{u'theme': {u'id': 22, u'name': u'Master Stor...",2014-07-03 20:43:21,537615,0
3,58,3,70,258,1557,9,114,56,394,54,...,en,2008-02-22 01:16:00,2764,2002-02-02,2014-01-13 21:43:12,[198],"[{u'theme': {u'id': 7, u'name': u'Presentation...",2015-03-31 10:48:45,712292,1
4,1249,34,337,1069,73,389,261,1785,1543,95,...,en,2007-04-06 17:51:00,7153,2003-02-02,2013-09-05 16:59:08,[99],"[{u'theme': {u'id': 1, u'name': u'The Creative...",2014-08-23 12:17:41,3200918,0


###Here also generate a feature to record multi-speaker talks 

In [68]:
onespeaker = talks_test.speaker_ids.apply(lambda x:len(x)==1)

In [69]:
sum(~onespeaker)

5

In [70]:
talks_test['onespeaker']=onespeaker

In [72]:
talks_test.to_json('test2.json')

In [73]:
talks_test.columns

Index([           u'Beautiful',            u'Confusing',
                 u'Courageous',          u'Fascinating',
                      u'Funny',          u'Informative',
                  u'Ingenious',            u'Inspiring',
               u'Jaw-dropping',           u'Longwinded',
                         u'OK',            u'Obnoxious',
                 u'Persuasive',         u'Unconvincing',
            u'commented_count',          u'description',
              u'emailed_count',             u'event_id',
                         u'id',             u'keywords',
                  u'languages',                 u'name',
       u'native_language_code',         u'published_at',
               u'rating_count',          u'recorded_at',
                u'released_at',          u'speaker_ids',
                     u'themes',           u'updated_at',
               u'viewed_count',       u'speaker_gender',
                 u'onespeaker'],
      dtype='object')

##2. Process talk text information: title + keywords + description

This is the first of all text resources that will be tapped on, the second would be the captions (e.g., first and last paragraphs); the third will be speakers' background info

In [74]:
talks_test = pd.read_json('test2.json')

In [75]:
labels = pd.read_csv('test_response.csv',header =None )

In [76]:
labels.shape

(350, 2)

In [77]:
labels.columns = ['id','label']

In [78]:
talks = pd.merge(talks_test[['description',
                             'id', 'keywords', 'name']],labels,on = 'id')

In [79]:
talks.sort(inplace = True, columns = 'id')

In [80]:
talks.reset_index(inplace = True, drop = True)

In [81]:
def gettitle(name):
    tmp = name.split(':')
    if len(tmp)==1:
        return tmp[0]
    else:
        return tmp[1]

In [82]:
talks['title'] = talks.name.apply(gettitle)

In [83]:
talks.drop('name',axis = 1,inplace = True)

In [84]:
talks.head()

,description,id,keywords,label,title
0,American designer Chris Bangle explains his ph...,5,"The Creative Spark,Tales of Invention,TED Conf...",3,Great cars are great art
1,In this emotionally charged conversation with ...,8,"The Power of Cities,Architectural Inspiration,...",3,A memorial at Ground Zero
2,"Jane Goodall hasn't found the missing link, bu...",11,"Animals That Amaze,Evolution's Genius,Inspired...",1,What separates us from chimpanzees?
3,In this inspiring talk about recent developmen...,18,"Animals That Amaze,Evolution's Genius,Inspired...",2,Biomimicry's surprising lessons from nature's...
4,"How many poets could cram eBay, Friendster and...",26,"Presentation Innovation,Spectacular Performanc...",4,If I controlled the Internet


In [85]:
text = []
for ind, row in talks.iterrows():
    text.append(' '.join([row['description'],row['keywords'],row['title']]))

In [86]:
talks['text'] = text

In [87]:
talks.text[0]

u"American designer Chris Bangle explains his philosophy that car design is an art form in its own right, with an entertaining -- and ultimately moving -- account of the BMW Group's Deep Blue project, intended to create the SUV of the future. The Creative Spark,Tales of Invention,TED Conference,arts,business,cars,design,industrial design,invention,technology,transportation  Great cars are great art"

###2.1 Remove tags  

In [88]:
talks['text'] = talks['text'].apply(lambda x: re.sub("<.*?>", "", x))    

In [89]:
talks.text[0]

u"American designer Chris Bangle explains his philosophy that car design is an art form in its own right, with an entertaining -- and ultimately moving -- account of the BMW Group's Deep Blue project, intended to create the SUV of the future. The Creative Spark,Tales of Invention,TED Conference,arts,business,cars,design,industrial design,invention,technology,transportation  Great cars are great art"

In [90]:
talks.text.shape

(350,)

In [91]:
talks.columns

Index([u'description', u'id', u'keywords', u'label', u'title', u'text'], dtype='object')

In [92]:
talks[['id','text']].to_json('validset_other_text.json')